# PRD: Market Research for Coffee Capsules v2.0

## Objective  
Assess the market opportunity for coffee capsules—an emerging coffee product format—by analyzing market size, key players, target audience, and competitive positioning versus other coffee types.

You need have a plan for the research. It includes:
1. Search related companies and products.
2. Summary of findings.
3. Generate the report as a html file.

## Scope  
The research must include:

- **Market Size**: Current global/regional market value and volume, plus 3–5 year growth projections.  
- **Key Players**: Major brands (e.g., Nespresso, Keurig, Lavazza), their market share, pricing, and distribution.  


## Requirements  
- Use data from the last 18 months  
- Highlight sustainability aspects (e.g., recyclability, consumer concerns)  
- Focus on actionable insights for product and marketing teams

Let's start.
First step is to prepare the MCP tool.

In [1]:
# Lets prepare the tool at first

from course_code.common.utils import load_env, stdio_mcp
import os

from agentscope.agent import ReActAgent
from agentscope.memory import InMemoryMemory
from agentscope.mcp import  StdIOStatefulClient
from agentscope.tool import (
    Toolkit,
)
# prepare the env
# please ensure that you have set the environment variables in .env file
load_env()

# prepare the MCP client of WebSearch
stdio_json={
    "mcp-server-firecrawl": {
      "args": [
        "-y",
        "firecrawl-mcp"
      ],
      "command": "npx",
      "env": {
        "FIRECRAWL_API_KEY": os.environ.get("FIRECRAWL_API_KEY","")
      }
    }
  }

search_mcp_client = StdIOStatefulClient(
    **stdio_mcp(stdio_json)
)
await search_mcp_client.connect()

2025-11-04 19:45:51,558 | INFO    | _stateful_client_base:connect:66 - MCP client connected.


Second step is build the plan.

In [2]:
from agentscope.plan import PlanNotebook, SubTask

plan_notebook = PlanNotebook()
await plan_notebook.create_plan(
    name="Market Research",
    description="Take a deep research on coffee capsules",
    expected_outcome="A markdown report to figure the market size and key players",
    subtasks=[
        SubTask(
            name="Search related companies and products.",
            description=(
                "Search related companies and products from 2024 to 2025"
            ),
            expected_outcome="Content in Markdown",
        ),
        SubTask(
            name="Summary of findings.",
            description="Summarize the content, find out the market size and key players",
            expected_outcome="Content Markdown",
        ),
        SubTask(
            name="Generate the report.",
            description=(
                "Generate the full report with three sections: 1. Intro coffee capsules; 2. Market size; 3. key players"
            ),
            expected_outcome="Final report in Markdown",
        )
    ],
)

ToolResponse(content=[{'type': 'text', 'text': "Plan 'Market Research' created successfully."}], metadata=None, stream=False, is_last=True, is_interrupted=False, id='2025-11-04 19:45:51.597_779eb0')

Third step is to build the agent.

In [3]:
# construct the prompt according to the PRD.
system_prompt="""You are a senior product researcher"""
prompt ="""
"""

In [4]:
from course_code.common.utils import markdown_to_html
from agentscope.formatter import OpenAIChatFormatter, DashScopeChatFormatter
from agentscope.message import Msg
from agentscope.model import OpenAIChatModel, DashScopeChatModel
from course_code.common.constants import MODELSCOPE_ENDPOINT,QWEN_MODEL_NAME
# construct the toolkit
toolkit = Toolkit()
# register the search mcp client in toolkit
await toolkit.register_mcp_client(
    search_mcp_client
)

# build the agent in the ReAct
agent = ReActAgent(
    # The name of the Agent
    name="MiMCoffeeResearcher",
    # The system prompt we prepared before.
    sys_prompt=system_prompt,
    # An OpenAI-compatible model
    model=DashScopeChatModel(
        # The API key of the ModelScope
        api_key=os.environ.get("DASHSCOPE_API_KEY"),
        # The model name
        model_name="qwen-max",
        # The args to declare the Endpoint

        stream=False,
    ),
    formatter=DashScopeChatFormatter(),
    plan_notebook=plan_notebook,
    toolkit=toolkit,
    memory=InMemoryMemory(),
)

# run the agent
reply = await agent( Msg("User",prompt,role="user"))

2025-11-04 19:45:51,750 | INFO    | _toolkit:register_mcp_client:730 - Registered 6 tool functions from MCP: firecrawl_scrape, firecrawl_map, firecrawl_search, firecrawl_crawl, firecrawl_check_crawl_status, firecrawl_extract.


MiMCoffeeResearcher: {
    "type": "tool_use",
    "name": "update_subtask_state",
    "input": {
        "subtask_idx": 0,
        "state": "in_progress"
    },
    "id": "call_b8716146967b48bb806f3f"
}
MiMCoffeeResearcher: {
    "type": "tool_use",
    "name": "firecrawl_search",
    "input": {
        "query": "related companies and products for coffee capsules",
        "limit": 5,
        "sources": [
            {
                "type": "web"
            }
        ]
    },
    "id": "call_8b97f8c8cbf24095b928c2"
}
system: {
    "type": "tool_result",
    "id": "call_b8716146967b48bb806f3f",
    "name": "update_subtask_state",
    "output": [
        {
            "type": "text",
            "text": "Subtask at index 0, named 'Search related companies and products.' is marked as 'in_progress' successfully. The plan state has been updated to 'in_progress'."
        }
    ]
}
system: {
    "type": "tool_result",
    "id": "call_8b97f8c8cbf24095b928c2",
    "name": "firecrawl_search

In [5]:
print(reply.content)

I have found some related companies and products for coffee capsules. Here are the top search results:

1. [Top 10 Coffee Pod Manufacturers Worldwide - Saneu](https://coffeemachinekz.com/top-10-coffee-pod-manufacturers-worldwide/)
   - Nestlé's Nespresso coffee brand is as well-known around the world as Coca-Cola. Nestlé invented K-cup, coffee capsules, Dolce Gusto and other...

2. [9 Best Coffee Capsules Manufacturers | Private Label, Bulk, Custom](https://usetorg.com/category/coffee-capsules)
   - The 9 Best Coffee Capsules Manufacturers in the World · REMACAFFE LTD. · Carmine rondinelli instant drink and coffee · Cafes baque · A. Kuster Sirocco AG.

3. [Which types of coffee capsules are the most popular?](https://perfectdailygrind.com/?p=101614)
   - Out of all the coffee capsule brands available on the market, Nespresso still remains the most popular by far. It's estimated that a staggering...

4. [Is there any other brand that is even better than nespresso capsule...](https://www